## Day 14: Disk Defragmentation

http://adventofcode.com/2017/day/14

### Part 1

This is reasonably straightforward, if a but fiddly. Reuse the code from <a href='https://github.com/mratford/advent/blob/master/2017/10/Knot%20Hash.ipynb'>Day 10</a> to calculate knot hashes.

In [1]:
from knot_hash import knot_hash

In [2]:
def hash_to_binary(h):
    return ''.join([bin(int(c, 16))[2:].zfill(4) for c in h])

In [3]:
hash_to_binary('a0c2017')

'1010000011000010000000010111'

According to the problem statement this should be `'10100000110000100000000101110000'` but the input string would then have to end in a zero so let's plough on.

In [4]:
from pyrsistent import pvector

def keys(base_key, number=128):
    for i in range(number):
        yield base_key + '-' + str(i)
        
def bit_grid(key): 
    return pvector(hash_to_binary(knot_hash(k)) for k in keys(key))

def squares(key):
    return sum(int(c) for c in ''.join(bit_grid(key)))

In [5]:
test_key = 'flqrgnkx'

%time assert squares(test_key) == 8108

CPU times: user 7.73 s, sys: 56 µs, total: 7.73 s
Wall time: 7.73 s


Too slow, but let's get the answer.

In [6]:
input_key = 'ugkiagan'

squares(input_key)

8292

### Part 2

Ooh, nice problem. I'll come back to it when I have more time. Here's a utility function for now.

In [7]:
def neighbours(x, y, max_x=127, max_y=127):
    for d in (-1, 1):
        if 0 <= x + d <= max_x:
            yield (x + d, y)
        if 0 <= y + d <= max_y:
            yield (x, y + d)

Sudden realisation that NetworkX will simplify this.

In [8]:
import networkx as nx

def active(bg, x, y):
    return bg[y][x] == '1'

def active_neighbours(bg, x, y, max_x=127, max_y=127):
    for n_x, n_y in neighbours(x, y, max_x, max_y):
        if active(bg, x, y) and active(bg, n_x, n_y):
            yield (n_x, n_y)

def bitgrid_to_graph(bg):
    max_x = len(bg[0]) - 1
    max_y = len(bg) - 1
    
    g = nx.Graph()
    
    for x in range(max_x + 1):
        for y in range(max_y + 1):
            if active(bg, x, y):
                g.add_node((x, y))
            for n_x, n_y in active_neighbours(bg, x, y, max_x, max_y):
                g.add_edge((x, y), (n_x, n_y))
                
    return g       

def number_of_regions(key):
    bg = bit_grid(key)
    g = bitgrid_to_graph(bg)
    return nx.number_connected_components(g)

That's pretty horrific.

In [9]:
assert number_of_regions(test_key) == 1242

That took some debugging. I'll leave some of it here for posterity.

In [10]:
from collections import defaultdict

def graph_to_bitgrid(g, max_x=127, max_y=127):
    d = defaultdict(lambda: '0')

    for x, y in g.nodes:
        d[(x, y)] = '1'
        
    return [''.join(d[(x, y)] for x in range(max_x + 1)) for y in range(max_y + 1)]

In [12]:
test_bg = bit_grid(test_key)

bg_to_g_to_bg = graph_to_bitgrid(bitgrid_to_graph(test_bg))

In [13]:
assert test_bg == bg_to_g_to_bg

In [14]:
def diff_bg(bg1, bg2):
    max_x = len(bg1[0]) - 1
    max_y = len(bg1) - 1

    for x in range(max_x + 1):
        for y in range(max_y + 1):
            if bg1[y][x] != bg2[y][x]:
                print(x, y, bg1[y][x], bg2[y][x])

In [15]:
diff_bg(test_bg, bg_to_g_to_bg)

In [16]:
number_of_regions(input_key)

1069